# Model training
---
Describe:

---

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import pickle
import model_config
from utils.time_utils import current_time

from catboost import CatBoostRegressor
# from bayes_opt import BayesianOptimization


from sklearn.model_selection import train_test_split
from catboost import cv, Pool

import feature_engineering_config as fe_config  
from feature_engineering import Standardization

## 2. Config

In [2]:
random_state = model_config.random_state

## 3. Load Data

In [3]:
with open(f'../Data/meta/train_x_after_feature_engineering.pickle', 'rb') as f:
    train_x = pickle.load(f)

In [4]:
with open(f'../Data/meta/train_y_after_feature_engineering.pickle', 'rb') as f:
    train_y = pickle.load(f)

In [5]:
# train_x, val_x, train_y, val_y = train_test_split(x_df, y_df, test_size=0.33, random_state=random_state)

In [6]:
if fe_config.standardization == 'standard_scaler':
    scaled_train_y = Standardization(train_y, 'y').standard_scaler(train=fe_config.train_process)
elif fe_config.standardization == 'min_max_scaler':
    scaled_train_y = Standardization(train_y, 'y').min_max_scaler(train=fe_config.train_process)

In [7]:
train_y.head()

,sensor_point5_i_value,sensor_point6_i_value,sensor_point7_i_value,sensor_point8_i_value,sensor_point9_i_value,sensor_point10_i_value
0,42,56,82,41,73,45
1,37,60,58,40,64,42
2,35,66,62,39,86,45
3,37,58,58,42,52,39
4,57,78,62,51,64,65


In [8]:
scaled_train_y.head()

,sensor_point5_i_value,sensor_point6_i_value,sensor_point7_i_value,sensor_point8_i_value,sensor_point9_i_value,sensor_point10_i_value
0,-2.810878,-1.358789,-1.088204,-1.693044,-0.745150,-2.660662
1,-3.189228,-1.209614,-2.015988,-1.737647,-1.195625,-2.830573
2,-3.340568,-0.985851,-1.861358,-1.782250,-0.094464,-2.660662
3,-3.189228,-1.284201,-2.015988,-1.648441,-1.796259,-3.000484
4,-1.675828,-0.538327,-1.861358,-1.247015,-1.195625,-1.527922


In [9]:
train_x.head()

,clean_pressure11,clean_pressure23,clean_pressure31,clean_pressure41,clean_pressure51,clean_pressure52,clean_pressure61,clean_pressure62,clean_pressure72,clean_pressure81,...,painting_g4_act_hvv_group,painting_g7_act_hvv_group,painting_g8_act_a_air_group,painting_g8_act_t_air_group,painting_g9_act_t_air_group,painting_g10_act_hvc_group,painting_g11_act_hvc_group,env_rpi05_temp_group,env_rpi07_hum_group,env_rpi15_pm1_group
0,0.996495,-2.393281,-0.839655,2.154155,2.111229,-2.887918,1.884733,-0.765608,2.854976,1.918546,...,2,3,0,4,1,3,1,1,3,2
1,0.994979,-2.393054,-0.840277,2.154155,2.120080,-2.887918,1.884733,-0.765809,2.854976,1.933153,...,2,3,0,1,1,3,1,1,3,2
2,0.994979,-2.393054,-0.840277,2.154155,2.120080,-2.887918,1.884733,-0.765809,2.854976,1.933153,...,2,3,0,1,1,3,1,1,3,2
3,0.994979,-2.393054,-0.840277,2.154155,2.120080,-2.887918,1.884733,-0.765809,2.854976,1.933153,...,2,3,0,1,1,3,1,1,3,2
4,1.005596,-2.395557,-0.850103,2.154155,2.115652,-2.886876,1.884733,-0.774647,2.849485,1.792013,...,2,3,0,1,1,3,1,1,3,2


In [10]:
categorical_features_indices = np.where(train_x.dtypes != float)[0]

## 4. Training model on each target variable

In [11]:
# def print_cv_summary(cv_data):
#     cv_data.head(10)

#     best_value = cv_data['test-RMSE-mean'].min()
#     best_iter = cv_data['test-RMSE-mean'].values.argmin()

#     print('Best validation Logloss score : {:.4f}±{:.4f} on step {}'.format(
#         best_value,
#         cv_data['test-Logloss-std'][best_iter],
#         best_iter)
#     )

In [12]:
# col_y = 'sensor_point5_i_value'
# model_dic = {}
# for col_y in train_y.columns:
    
#     params = {
#         'loss_function': 'RMSE',
#         'iterations': 80,
#         'random_seed': random_state,
#         'learning_rate': 0.5,
#         # 'use_best_model': True
#     }

#     cv_data = cv(
#         params = params,
#         pool = Pool(train_x, label=train_y[col_y], cat_features=categorical_features_indices),
#         fold_count=5,
#         type = 'Classical',  # The method to split the dataset into folds.
#         shuffle=True,
#         partition_random_seed=random_state,
#         plot=False,
#         stratified=False,
#         verbose=False
#     )

#     print_cv_summary(cv_data)


#     best_value = np.argmin(cv_data['test-RMSE-mean'])
#     best_iter = np.argmin(cv_data['test-RMSE-mean'])

#     print('Best validation RMSE score: {:.4f}±{:.4f} on step {}'.format(
#         best_value,
#         cv_data['test-RMSE-std'][best_iter],
#         best_iter)
#     )

#     params['iterations'] = best_iter

#     model = CatBoostRegressor(
#         **params,
#         eval_metric='RMSE',
#         verbose=False
#     )

#     model.fit(
#         train_x, train_y[col_y],
#         cat_features=categorical_features_indices,
#     #     eval_set=(val_x, val_y[col_y]),
#         # logging_level='Verbose',
#         plot=False
#     )
    
#     model_dic[col_y] = model

In [13]:
model_dic = {}
for col_y in scaled_train_y.columns:

    params = {
        'loss_function': 'RMSE',
        'random_seed': random_state,
        'early_stopping_rounds':100,
        'task_type': 'CPU',
        'verbose':False,
        'has_time':True,
    }
    model = CatBoostRegressor(**params)

    grid = {
        'learning_rate': [0.05, 0.07, 0.09, 0.3],
        'iterations': [50, 60, 70, 80, 90, 100, 120, ],
        'depth': [3, 4, 5, 6, 7],
        'l2_leaf_reg': [1, 3, 5, 7, 9],
        'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide']
    }
    randomized_search_result = model.randomized_search(
        grid,
        X=train_x,
        y=scaled_train_y[col_y],
        cv=3,
        plot=False
    )
    
    model_dic[col_y] = {'model': model, 'randomized_search_result': randomized_search_result}


bestTest = 0.6835355772
bestIteration = 99

0:	loss: 0.6835356	best: 0.6835356 (0)	total: 223ms	remaining: 2.01s

bestTest = 0.6791751367
bestIteration = 76

1:	loss: 0.6791751	best: 0.6791751 (1)	total: 459ms	remaining: 1.84s

bestTest = 0.6570805584
bestIteration = 34

2:	loss: 0.6570806	best: 0.6570806 (2)	total: 553ms	remaining: 1.29s

bestTest = 0.6826395825
bestIteration = 99

3:	loss: 0.6826396	best: 0.6570806 (2)	total: 683ms	remaining: 1.02s

bestTest = 0.6681433793
bestIteration = 58

4:	loss: 0.6681434	best: 0.6570806 (2)	total: 972ms	remaining: 972ms

bestTest = 0.6656640975
bestIteration = 79

5:	loss: 0.6656641	best: 0.6570806 (2)	total: 1.74s	remaining: 1.16s

bestTest = 0.6889470886
bestIteration = 89

6:	loss: 0.6889471	best: 0.6570806 (2)	total: 1.83s	remaining: 785ms

bestTest = 0.6827262231
bestIteration = 35

7:	loss: 0.6827262	best: 0.6570806 (2)	total: 2.08s	remaining: 521ms
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.6703445857
bestIter

In [14]:
# model_dic[col_y]['model'].predict(train_x)

In [15]:
# model_dic[col_y]['randomized_search_result']['params']

In [16]:
# model_dic[col_y]['randomized_search_result']['cv_results']['test-RMSE-mean']

In [17]:
def rmse_score(predictions, targets):
    return np.sqrt(np.mean((predictions-targets)**2,  axis=0))

In [18]:
array_li= [model_dic[col_y]['model'].predict(train_x).reshape(-1,1) for col_y in scaled_train_y.columns]
all_res = np.concatenate(array_li, axis=1)

In [20]:
with open(f'../Model/standard_scaler_y.pickle', 'rb') as f:
    scaler_y = pickle.load(f)

In [21]:
predictions = scaler_y.inverse_transform(all_res)
targets = train_y
print(rmse_score(predictions, targets))

sensor_point5_i_value      6.201924
sensor_point6_i_value      7.934518
sensor_point7_i_value     10.532128
sensor_point8_i_value      8.252567
sensor_point9_i_value     12.141868
sensor_point10_i_value     8.492629
dtype: float64


In [22]:
time = current_time()
for col_y in scaled_train_y.columns:
    with open(f'../Model/catboost_{col_y}_{time}.pickle', 'wb' ) as f:
        pickle.dump(model_dic[col_y]['model'], f)

## 5. Catboost for Muti-Output 

In [23]:
params = {
    'loss_function': 'MultiRMSE',
    'random_seed': random_state,
    'early_stopping_rounds':100,
    'task_type': 'CPU',
    'verbose':False,
    'has_time':True,
}
model = CatBoostRegressor(**params)

grid = {
    'learning_rate': [0.05, 0.07, 0.09, 0.3],
    'iterations': [50, 60, 70, 80, 90, 100],
    'depth': [3, 4, 5, 6, 7],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide']
}

randomized_search_result = model.randomized_search(
    grid,
    X=train_x,
    y=scaled_train_y,
    cv=3,
    plot=False
)
model_muti_output_dic = {'model': model, 'randomized_search_result': randomized_search_result}


bestTest = 1.777835741
bestIteration = 79

0:	loss: 1.7778357	best: 1.7778357 (0)	total: 97ms	remaining: 873ms

bestTest = 1.67625376
bestIteration = 31

1:	loss: 1.6762538	best: 1.6762538 (1)	total: 218ms	remaining: 871ms

bestTest = 1.709499214
bestIteration = 69

2:	loss: 1.7094992	best: 1.6762538 (1)	total: 437ms	remaining: 1.02s

bestTest = 1.699149477
bestIteration = 59

3:	loss: 1.6991495	best: 1.6762538 (1)	total: 1.05s	remaining: 1.58s

bestTest = 1.647520381
bestIteration = 62

4:	loss: 1.6475204	best: 1.6475204 (4)	total: 1.67s	remaining: 1.67s

bestTest = 1.672124256
bestIteration = 84

5:	loss: 1.6721243	best: 1.6475204 (4)	total: 2.32s	remaining: 1.55s

bestTest = 1.729616098
bestIteration = 99

6:	loss: 1.7296161	best: 1.6475204 (4)	total: 2.56s	remaining: 1.1s

bestTest = 1.679557235
bestIteration = 58

7:	loss: 1.6795572	best: 1.6475204 (4)	total: 2.9s	remaining: 724ms

bestTest = 1.735723559
bestIteration = 99

8:	loss: 1.7357236	best: 1.6475204 (4)	total: 3.17s	rema

In [24]:
predictions = model_muti_output_dic['model'].predict(train_x)
targets = scaled_train_y
print(rmse_score(predictions, targets))

sensor_point5_i_value     0.500922
sensor_point6_i_value     0.372831
sensor_point7_i_value     0.498680
sensor_point8_i_value     0.456798
sensor_point9_i_value     0.519749
sensor_point10_i_value    0.572766
dtype: float64


In [25]:
predictions = scaler_y.inverse_transform( model_muti_output_dic['model'].predict(train_x) )
targets = train_y
print(rmse_score(predictions, targets))

sensor_point5_i_value      6.619830
sensor_point6_i_value      9.997157
sensor_point7_i_value     12.899905
sensor_point8_i_value     10.241432
sensor_point9_i_value     10.384013
sensor_point10_i_value    10.112925
dtype: float64


In [ ]:
# from sklearn.multioutput import MultiOutputRegressor
# from sklearn.model_selection import RepeatedKFold
# from sklearn.model_selection import cross_val_score

In [ ]:
# params = {
#     'loss_function': 'MultiRMSE',
#     'iterations': 150,
#     'random_seed': random_state,
#     'learning_rate': 0.5,
#     # 'use_best_model': True
# }

# cv_data = cv(
#     params = params,
#     pool = Pool(train_x, label=train_y, cat_features=categorical_features_indices),
#     fold_count=5,
#     type = 'Classical',  # The method to split the dataset into folds.
#     shuffle=True,
#     partition_random_seed=random_state,
#     plot=True,
#     stratified=False,
#     verbose=False
# )

In [ ]:
# best_value = np.argmin(cv_data['test-MultiRMSE-mean'])
# best_iter = np.argmin(cv_data['test-MultiRMSE-mean'])

# print('Best validation MultiRMSE score: {:.4f}±{:.4f} on step {}'.format(
#     best_value,
#     cv_data['test-MultiRMSE-std'][best_iter],
#     best_iter)
# )

In [ ]:
# params['iterations'] = best_iter

In [ ]:
# model = CatBoostRegressor(
#     **params,
#     eval_metric='MultiRMSE',
#     verbose=False
# )

# model.fit(
#     train_x, train_y,
#     cat_features=categorical_features_indices,
#     eval_set=(val_x, val_y),
#     # logging_level='Verbose',
#     plot=False
# )

In [ ]:
# model.get_evals_result()

In [ ]:
# predict_y = model.predict(val_x)

In [ ]:
# def rmse_score(predictions, targets):
#     return np.sqrt(np.mean((predictions-targets)**2, axis=0))

In [ ]:
# rmse_score(predict_y, val_y)